In [12]:
import sys

sys.path.append("/Users/thomas.cong/Downloads/ResearchCode")
import pandas as pd
from feature_calculation import build_biomarker_csv
import warnings

warnings.filterwarnings("ignore")

In [2]:
build_biomarker_csv.build_csv(
    "./Park_Celeb_Biomarkers.csv",
    audio_dir="/Users/thomas.cong/Downloads/ResearchCode/ParkCelebCode/Preprocessed Park Celeb Files",
    results_dir="./Park_Celeb_Biomarkers",
)

Processing audio files:   7%|▋         | 91/1289 [02:06<43:27,  2.18s/it]  

Audio file pd_31_90mj79GqWhc_9 is too short, skipping...


Processing audio files:   7%|▋         | 95/1289 [02:17<34:09,  1.72s/it]  

Audio file pd_24_xpIYmtHT_7k_7 is too short, skipping...


Processing audio files:  11%|█         | 141/1289 [03:18<23:33,  1.23s/it]

...

Audio file cn_29_oTdDV_ZMCWg_-12 is too short, skipping...


Processing audio files:  96%|█████████▌| 1237/1289 [36:27<00:48,  1.07it/s]

Audio file pd_35_019ZO8Mj3ws_3 is too short, skipping...


Processing audio files: 100%|██████████| 1289/1289 [37:57<00:00,  1.77s/it]


Metrics saved to ./Park_Celeb_Biomarkers.csv


In [13]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")
ParkCelebBiomarkers.drop_duplicates(inplace=True)


def get_yfd(filename):
    try:
        return float(filename.split("_")[-1])
    except:
        return None


ParkCelebBiomarkers["YFD"] = (
    ParkCelebBiomarkers.index.str.split("_").str[-1].map(get_yfd)
)
ParkCelebBiomarkers["Status"] = (
    ParkCelebBiomarkers.index.str.split("_").str[0].str.upper()
)
ParkCelebBiomarkers.to_csv("./Park_Celeb_Biomarkers_New.csv")

In [14]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")
ParkCelebBiomarkers["subjid"] = (
    ParkCelebBiomarkers.index.str.split("_").str[0:2].str.join("_")
)
ParkCelebBiomarkers.to_csv("./Park_Celeb_Biomarkers.csv")

In [ ]:
ParkCelebBiomarkers = pd.read_csv("./Park_Celeb_Biomarkers.csv", index_col="filename")
first_and_last = []
firsts = []
lasts = []
for subjid in ParkCelebBiomarkers["subjid"].unique():
    first = ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid].YFD.min()
    last = ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid].YFD.max()
    first_indices = ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid][
        ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid]["YFD"] == first
    ].index
    last_indices = ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid][
        ParkCelebBiomarkers[ParkCelebBiomarkers["subjid"] == subjid]["YFD"] == last
    ].index
    first_and_last.extend(first_indices)
    first_and_last.extend(last_indices)
    firsts.extend(first_indices)
    lasts.extend(last_indices)
first_and_last_df = ParkCelebBiomarkers.loc[first_and_last]
first_and_last_df["First/Last"] = first_and_last_df.index.map(lambda x: "First" if x in firsts else "Last")
first_and_last_df.to_csv("./Park_Celeb_Biomarkers_First_Last.csv")

In [17]:
first_indices

Index(['pd_21_EPJIWzFCANs_10', 'pd_21_A25tlEGQNu4_10'], dtype='object', name='filename')